<a href="https://colab.research.google.com/github/sanjaydasgupta/LLM-created-patent-database/blob/main/few_shot_trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting Patent Bibliographic Summaries to JSON Records

**CAUTION**: This notebook needs a T4 GPU, and runs for 25-30 minutes. Running it may incur a substantial cost.

## Install Essential Libraries

In [ ]:
!pip install -q -U flash-attn
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U accelerate

## Load a Model

**IMPORTANT**: Obtain a [huggingface token](https://huggingface.co/docs/hub/en/security-tokens) and save it in the Secrets panel on the left, using the name "HF_TOKEN". Then restart the run-time before running the cell below.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

################### Un-comment one of the folowing LLM names #############################
#
# model_id = "microsoft/Phi-3-mini-128k-instruct"
# model_id = "microsoft/Phi-3-mini-4k-instruct"
#
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#
################### The following LLMs do not support system-prompts #############################
# model_id = "mistralai/Mistral-7B-Instruct-v0.2"
# model_id = "google/gemma-7b-it"
# model_id = "google/gemma-1.1-7b-it"
#

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Run Predictions

**IMPORTANT**: Add your HF_TOKEN ([huggingface token](https://huggingface.co/docs/hub/en/security-tokens)) in the Secrets panel on the left before running the cell below.

The following cell uses a dataset (*s11a.parquet* in the `data` folder) containing two columns *input* and *output*. A few rows of this dataset are used for setting up the few-shot prompting examples. Other rows are used for testing the accuracy of the prompted LLMs conversions. The column *input* contains unstructured text extracted from pages of volume II of the [Official Journal of the Patent Office](https://search.ipindia.gov.in/IPOJournal/Journal/Patent) published on 19th April 2024. The column *output* contains corresponding structured text represented as JSON strings. The strings in *input* were obtained by using [PyMuPDF](https://pypi.org/project/PyMuPDF/). The JSON strings in *output* were obtained using a process completely different from that described here. The dataset contains over 690 rows -- one row for each page of the journal. The PDF document from which the *input* column was created can be found in the `data` folder.

In [ ]:
import pandas as pd
from datetime import datetime
from difflib import SequenceMatcher

prompt = """Study the following pairs of Input (plain-text) and Output (JSON) examples.
Then convert the user-provided plain-text record into JSON.
Each input contains multiple SECTIONs; and each SECTION contains a TAG, a DESCRIPTION, and a VALUE.
The following TAG-DESCRIPTION combinations are used:
"(21) Application No", "(19)", "(22) Date of filing of Application", "(43) Publication Date",
"(54) Title of the invention", "(51) International classification", "(31) Priority Document No", "(32) Priority Date",
"(33) Name of priority country",  "(86) International Application No", "(87) International Publication No ",
"(61) Patent of Addition to Application Number", "(62) Divisional to Application Number", "(71)Name of Applicant",
"(72)Name of Inventor", and "(57) Abstract".
Minor typos or small textual differences can occur in the DESCRIPTION part.
In the Input, the SECTIONs may come in any order, but the SECTION starting with "(57) Abstract" always comes at the end.
From each SECTION, extract the TAG and the VALUE to produce a TAG-VALUE pair.
A VALUE that contains just a date must be rewritten in YYYY-MM-DD (ISO-8601) style.
A VALUE that contains just a country or region name must be rewritten as a 2-character (ISO-3166) code.
Combine all such TAG-VALUE pairs into a JSON Response with the TAGs arranged in ascending order.
Do not generate any TAG-VALUE pairs that are not found in the Input."""

def score_prediction(output: str, prediction: str) -> float:
    return SequenceMatcher(None, output, prediction).ratio()

test_row_count = 20 # Number of rows for testing
few_shot_count = 5 # Number of rows reserved for few-shot prompting

data_url = "https://github.com/sanjaydasgupta/LLM-created-patent-database/raw/main/data/s11a.parquet"
df5 = pd.read_parquet(data_url).sample(test_row_count + few_shot_count)
print('Row count:', len(df5), 'Columns:', list(df5), '\n')

total_elapsed_seconds = total_score = 0

for input, output in df5.tail(test_row_count).values:
    print('INPUT:', input)
    print('OUTPUT:', output)

    generation_args = {
        "max_new_tokens": 1024,
        "return_full_text": False,
        "temperature": 0.05,
        "do_sample": True,
    }

    messages = [
        # System prompt ...
        {"role": "system", "content": prompt},
        # Shot #1 input and output (from first and second columns of DataFrame) ...
        {"role": "user", "content": df5.iat[0, 0]},
        {"role": "assistant", "content": df5.iat[0, 1]},
        # Shot #2 input and output (from first and second columns of DataFrame) ...
        {"role": "user", "content": df5.iat[1, 0]},
        {"role": "assistant", "content": df5.iat[1, 1]},
        # Shot #3 input and output (from first and second columns of DataFrame) ...
        {"role": "user", "content": df5.iat[2, 0]},
        {"role": "assistant", "content": df5.iat[2, 1]},
        # Question for LLM ...
        {"role": "user", "content": input},
    ]

    t0 = datetime.now()
    prediction = pipe(messages, **generation_args)
    predicted_string = prediction[0]['generated_text'].strip()
    delta_seconds = (datetime.now() - t0).total_seconds()
    total_elapsed_seconds += delta_seconds
    score = score_prediction(output, predicted_string)
    total_score += score
    length_ratio = len(predicted_string) / len(output)
    print('PRED:', predicted_string)
    print('score:', score, 'length:', length_ratio, 'seconds:', delta_seconds)
    print('\n')

print('Model:', model_id, 'Mean time:', total_elapsed_seconds / test_row_count, 'Mean score:', total_score / test_row_count)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Row count: 25 Columns: ['input', 'output'] 

INPUT: (21) Application No.202327077326 A  (19) INDIA  (22) Date of filing of Application :14/11/2023  (43) Publication Date : 19/04/2024    (54) Title of the invention : DISCONTINUOUS RECEPTION METHODOLOGY FOR MULTIMEDIA     (51) International classification  :H04W 76/28  (31) Priority Document No  :63/203156  (32) Priority Date  :09/07/2021  (33) Name of priority country  :U.S.A.  (86) International Application No          Filing Date  :PCT/US2022/072846  :09/06/2022  (87) International Publication No  :WO 2023/283512  (61) Patent of Addition to Application Number          Filing Date  :NA :NA (62) Divisional to Application Number          Filing Date  :NA :NA    (71)Name of Applicant :      1)QUALCOMM INCORPORATED        Address of Applicant :ATTN: International IP Administration 5775 Morehouse Drive San Diego, California 92121 -1714 U.S.A.  (72)Name of Inventor :     1)LEE, Hyun Yong     2)HANDE, Prashanth Haridas     3)TINNAKORNSRISUPHA

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


PRED: {"(19)": "INDIA", "(21)": "202327077326 A", "(22)": "2023-11-14", "(31)": "63/203156", "(32)": "2021-07-09", "(33)": "US", "(43)": "2024-04-19", "(51)": "H04W 76/28", "(54)": "DISCONTINUOUS RECEPTION METHODOLOGY FOR MULTIMEDIA", "(57)": "Various aspects of the present disclosure generally relate to wireless communication. In some aspects, a user equipment (UE) may set a hyper frame length based at least  in part on a cadence of multimedia data bursts if discontinuous reception (DRX) cycle are configured for the cadence of the multimedia data bursts. The UE may receive the multimedia data bursts according to the DRX cycles. Numerous other aspects are descr ibed.     No. of Pages : 79  No. of Claims : 30", "(61)": "NA :NA", "(62)": "NA :NA", "(71)": "1)QUALCOMM INCORPORATED        Address of Applicant :ATTN: International IP Administration 5775 Morehouse Drive San Diego, California 92121 -1714 U.S.A.", "(72)": "1)LEE, Hyun Yong     2)HANDE, Prashanth Haridas     3)TINNAKORNSRISUPHA

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

PRED: {"(19)": "INDIA", "(21)": "202317011916 A", "(22)": "2023-02-22", "(31)": "202010785378.2", "(32)": "2020-08-06", "(33)": "CN", "(43)": "2024-04-19", "(51)": "H04W 4/40", "(54)": "RESOURCE AWARENESS METHOD AND COMMUNICATION APPARATUS", "(57)": "Embodiments of the present application provide a resource awareness method and a communication apparatus. The resource awareness method can comprise: a first terminal device determines, in a first selection window, a first candidate resource set used for sidelink transmission, and the first terminal device determines a second candidate resource set according to a resource aware ness result in a first awareness window, the second candidate resource set being a univer sal set or a subset of the first candidate resource set; and the first terminal device performs awareness on a sidelink resource in a second awareness window, and sends a resour ce awareness result in the second awareness window to a second terminal device at a first moment, th

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


PRED: {"(19)": "INDIA", "(21)": "202317012254 A", "(22)": "2023-02-23", "(31)": "10-2020 -0117862", "(32)": "2020-09-14", "(33)": "KR", "(43)": "2024-04-19", "(51)": "A61K 31/702, A61P 9/00, A61P 9/10, A61P 7/02, A61P 9/12", "(54)": "FOOD COMPOSITION FOR ALLEVIATING OR TREATING CARDIO -CEREBROVASCULAR DISEASE, AND PHARMACEUTICAL COMPOSITION FOR PREVENTING OR TREATING CARDIO -CEREBROVASCULAR DISEASE, COMPRISING 2' -FUCOSYL LACTOSE", "(57)": "The present invention relates to a 2' -fucosyllactose -containing food composition and pharmaceutical composition effective in alleviating, preventing, or treating vari ous cardio -cerebrovascular diseases caused by thrombus formation, and 2' -fucosyllactose (2' -FL) of the present invention exhibits agonistic (inhibitory) action with respect to collagen and collagen -related peptide (CRP), which is a platelet agonist, and th us may be used to suppress thrombus formation caused by abnormal action of platelets.    No. of Pages : 25  No. of Claims : 8

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


PRED: {"(19)": "INDIA", "(21)": "202324067856 A", "(22)": "2023-10-10", "(31)": "2022-164418", "(32)": "2022-10-13", "(33)": "JP", "(43)": "2024-04-19", "(51)": "H05B0041288000, H05B0041292 000, B60Q0001000000, H05B0041040000, H05B0041230000", "(54)": "DISCHARGE LAMP LIGHTING DEVICE AND EXPOSURE APPARATUS INCLUDING SAME", "(57)": "It is intended to eliminate an adverse effect exerted on an igniter by ultraviolet rays emit ted from a discharge lamp by disposing the igniter in adjacent to a power supply; namely, it is intended to provide a discharge lamp lighting device that eliminates nec essity to dispose the igniter in adjacent to the discharge lamp. The present discharge l amp lighting circuit (10), provided in a discharge lamp lighting device (A) for lighting a discharge lamp (L) of a direct -current lighting type, activates and lights the discharge lamp (L) through an extension cable (C) and is composed of a ballast", "(61)": "NA :NA", "(62)": "NA :NA", "(71)": "1)PHOENIX ELECTRIC 